# Pure Technical Strategy Backtest for Crypto

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from itertools import product
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

from ..utils.technical_indicators import *

In [ ]:
class PureTechnicalBacktest():

    def __init__(self,  symbol, start_date, trading_costs):
        self.symbol = symbol
        self.start_date = start_date
        self.trading_costs = trading_costs
        self.results = None
        self.data_df = self.get_data()
        self.tp_year = (self.data_df.close.count() / ((self.data_df.index[-1] - self.data_df.index[0]).days / 365.25))
        

    def __repr__(self):
        return "PureTechnicalBacktest(symbol = {}, start = {})".format(self.symbol, self.start_date)
    
    def get_data(self):
        
    
    def prepare_data(self):

        data = self.data.copy()

        # ******************** define your strategy here ************************
        data = relative_strength_index(data)
        data = macd(data)
        data = stochastic_oscillator(data)
        data = bollinger_bands(data)
        data = average_true_range(data)
        data = average_directional_index(data)

        data.dropna(inplace=True)

        adx_cond = data['adx'].iloc[-1] > 25

        # Buy Conditions
        bcond1 = (data['rsi'].shift(1) < 30) & (
            data['rsi'] > data['rsi'].shift(1))
        bcond2 = data['macd'] > data['signal']
        bcond3 = (data['%K'].shift(1) < data['%D'].shift(1)) & (
            data['%K'] < 20) & (data['%K'] > data['%D'])
        bcond4 = data['close'] <= data['lower_band']

        buy_cond = bcond1 & bcond2 & bcond3 & bcond4 & adx_cond

        # Sell Conditions
        scond1 = (data['rsi'].shift(1) > 70) & (
            data['rsi'] < data['rsi'].shift(1))
        scond2 = data['macd'] < data['signal']
        scond3 = (data['%K'].shift(1) > data['%D'].shift(1)) & (
            data['%K'] > 80) & (data['%K'] < data['%D'])
        scond4 = data['close'] >= data['upper_band']

        sell_cond = scond1 & scond2 & scond3 & scond4 & adx_cond

        data["position"] = 0
        data.loc[buy_cond, "position"] = 1
        data.loc[sell_cond, "position"] = -1

        self.results = data.copy()

    def run_backtest(self):

        data = self.results.copy()
        data['strategy'] = data.position.diff().fillna(0).abs()
        data['trades'] = data.position.diff().fillna(0)
        data["strategy"] = data["position"].shift(1) * data["returns"]
        data["trades"] = data.position.diff().fillna(0).abs()
        data.strategy = data.strategy + data.trades * self.trading_costs

        self.results = data

    def test_strategy(self, smas):

        self.prepare_data()
        self.run_backtest()
        
        data = self.results.copy()
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
        
        self.print_performance()

    def plot_results(self, leverage = False): #Adj!
        '''  Plots the cumulative performance of the trading strategy compared to buy-and-hold.
        '''
        if self.results is None:
            print("Run test_strategy() first.")
        elif leverage: 
            title = "{} | Trading Costs = {} | Leverage = {}".format(self.symbol, self.trading_costs, self.leverage)
            self.results[["creturns", "cstrategy", "cstrategy_levered"]].plot(title=title, figsize=(12, 8))
        else:
            title = "{} | Trading Costs = {}".format(self.symbol, self.trading_costs)
            self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))

    def print_performance(self, leverage = False): # Adj
        ''' Calculates and prints various Performance Metrics.
        '''
        
        data = self.results.copy()
        
        if leverage: # NEW!
            to_analyze = np.log(data.strategy_levered.add(1))
        else: 
            to_analyze = data.strategy
            
            
        strategy_multiple = round(self.calculate_multiple(to_analyze), 6)
        bh_multiple =       round(self.calculate_multiple(data.returns), 6)
        outperf =           round(strategy_multiple - bh_multiple, 6)
        cagr =              round(self.calculate_cagr(to_analyze), 6)
        ann_mean =          round(self.calculate_annualized_mean(to_analyze), 6)
        ann_std =           round(self.calculate_annualized_std(to_analyze), 6)
        sharpe =            round(self.calculate_sharpe(to_analyze), 6)
       
        print(100 * "=")
        print("Pure Technical| INSTRUMENT = {} |".format(self.symbol))
        print(100 * "-")
        print("PERFORMANCE MEASURES:")
        print("\n")
        print("Multiple (Strategy):         {}".format(strategy_multiple))
        print("Multiple (Buy-and-Hold):     {}".format(bh_multiple))
        print(38 * "-")
        print("Out-/Underperformance:       {}".format(outperf))
        print("\n")
        print("CAGR:                        {}".format(cagr))
        print("Annualized Mean:             {}".format(ann_mean))
        print("Annualized Std:              {}".format(ann_std))
        print("Sharpe Ratio:                {}".format(sharpe))
        
        print(100 * "=")

    def calculate_multiple(self, series):
        return np.exp(series.sum())
    
    def calculate_cagr(self, series):
        return np.exp(series.sum())**(1/((series.index[-1] - series.index[0]).days / 365.25)) - 1
    
    def calculate_annualized_mean(self, series):
        return series.mean() * self.tp_year
    
    def calculate_annualized_std(self, series):
        return series.std() * np.sqrt(self.tp_year)
    
    def calculate_sharpe(self, series):
        if series.std() == 0:
            return np.nan
        else:
            return self.calculate_cagr(series) / self.calculate_annualized_std(series)

